In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from datetime import datetime
import plotly.graph_objects as go
import pandas as pd

In [ ]:
cliwoc15 = pd.read_csv('archive/CLIWOC15.csv', low_memory=False)

In [ ]:
ship_name = cliwoc15['ShipName'].unique()[12]
df_ship = cliwoc15[cliwoc15['ShipName'] == ship_name]
df_ship = df_ship.reset_index()

print('Ship name = ', ship_name)

In [ ]:
# cliwoc15[~np.isnan(cliwoc15['WindForceScale'])]['ShipName'].unique()
# 'Neerlands Koning', 'Sara Johanna', 'Mentor', 'Soerabaya',
#        'Palembang', 'Rotterdam', 'Zaan', 'Maria', 'Rhoon en Pendrecht',
#        'Castor', 'Boreas', 'Heldin', 'Ambarawa', 'Prins Hendrik',
#        'Nieuw Lekkerland', 'Menado', 'Nehalennia', 'Unknown-01',
#        'Amicitia', 'Koning Willem II', 'Phoenix', 'Merapi', 'Celebes',
#        'Cornelia Sara', 'Prins Veldmaarschalk',
#        'Princes Royal Frederique', 'Willem', 'Sara Lydia',
#        'Albrecht Beijling', "'s-Gravenhage", 'Rotterdams Welvaren',
#        'Ternate', 'Magdalena', 'India', 'Unknown-02', 'Maria Elisabeth',
#        'Watergeus', 'Rhijn', 'Van Oldenbarneveld', 'Onverwagt', 'Johanna',
#        'Susanna', 'Stad Utrecht', 'Javaan', 'Theodora en Sara',
#        'Catharina Johanna', 'Neerlands Indië', 'Amstel', 'Buitenzorg',
#        'Ceres', 'Christiaan Huijgens', 'Claudius Civilis', 'Diana',
#        'Drie Vrienden', 'Egmond', 'Elisabeth', 'Eliza Susanna', 'Eolus',
#        'Hippomenes', 'Schorpioen', 'Wilhelmina Lucia', "Lucipara's",
#        'Admiraal Jan Evertsen', 'Margaretha Simonette', 'Jacoba',
#        'Maas Nijmph', 'Marij-Goddard', 'Merwede', 'Eliza',
#        'Olivier van Noord', 'Unknown-04', 'Unknown-05',
#        'Rembrandt van Rhijn', 'Margaretha en Maria', 'Unknown-06',
#        'China', 'Oost-Indië Packet', 'Straat Bali', 'Catharina',
#        'Unknown-07', 'Soolo', 'Witte Cornelisz de Wit', 'Jacob Roggeveen',
#        'Loopuijt', 'Oud Nederland', 'Unknown-08', 'Unknown-09',
#        'Hendrika', 'Anthony', 'Nederwaard', 'Cortgene', 'Europa',
#        'Proteus', 'Doctrina et Amicitia', 'Java Koerier',
#        'Gouverneur Elsevier', 'Amphitrite', 'Aurora', 'Staatsraad Baud',
#        'Haay', 'Walvisch', 'Argo', 'Unknown-10', 'Jacoba Maurina',
#        'Bellona', 'Nassau', 'Twee Cornelissen', 'Drie Gebroeders',
#        'Prinses Marianne', 'Henriette Clasina', 'Zeeuw',
#        'Prins Van Oranje', 'Jacobus', 'Jan Daniel', 'Stad Zierikzee',
#        'Azia', 'Plancius', 'Meermin', 'Sambre', 'Anna', 'Zwaan', 'Panter',
#        'Zwaluw', 'Maas', 'Unknown-12', 'Venus', 'Batavia', 'Mercator',
#        'Bromo', 'Estafette', 'Cornelia', 'Amsterdam']

# Select route with wind information 
# df_ship = cliwoc15[cliwoc15['ShipName'] == 'Marij-Goddard']

In [ ]:

# Remove nan and fill missing values
df_ship = df_ship[~np.isnan(df_ship['Lat3'])]
df_ship = df_ship[~np.isnan(df_ship['Lon3'])]
df_ship['WindForceScale'].fillna(0, inplace=True)
df_ship['TairReading'].fillna(0, inplace=True)
df_ship['WindForceScale'].fillna(0, inplace=True)

# Sort by date 
df_ship.sort_values(by=['UTC'], inplace=True)

# extract lat/lon 
lat = df_ship['Lat3']
lon = df_ship['Lon3']
coord = np.column_stack((list(lon),list(lat)))
y,x = coord[:,0], coord[:,1]

# convert UTC into date obj
utc = list(df_ship['UTC'])
fdate = [datetime.strptime(str(t)[6:8]+'.'+ str(t)[4:6]+'.'+str(t)[:4], '%d.%m.%Y') for t in utc]

# parameters
wforce = list(df_ship['WindForceScale'])
temperature = list(df_ship['TairReading'])
pressure = list(df_ship['BaroReading'])
weather = list(zip(df_ship['Rain'],df_ship['Thunder'],df_ship['Fog'],df_ship['Snow'],df_ship['Hail'],df_ship['SeaIce'],df_ship['Gusts']))

# create plots 
fig = make_subplots(
    rows=4, cols=2,
    specs=[[{"type": "scattergeo", "rowspan": 4}, {"type": "scatter"}],
    [None,{"type": "scatter"}],
    [None,{"type": "scatter"}],
    [None,{"type": "bar"}]])

# map with route 
fig.add_trace(go.Scattergeo(lat=x, lon=y, mode = 'lines', name='Route'),
    row=1, col=1
)

fig.add_trace(go.Scattergeo(lat=[x[0]], lon=[y[0]], showlegend=False),
    row=1, col=1
)



# Wind force
fig.add_trace(go.Scatter(x=[fdate[0]], y=[wforce[0]], showlegend=False),
    row=1, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=wforce, mode='lines', name='Wind force'),
    row=1, col=2
)

# Temperature
fig.add_trace(go.Scatter(x=[fdate[0]], y=[temperature[0]], showlegend=False),
    row=2, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=temperature, mode='lines', name='Temperature'),
    row=2, col=2
)


# Pressure
fig.add_trace(go.Scatter(x=[fdate[0]], y=[pressure[0]], showlegend=False),
  row=3, col=2
)

fig.add_trace(go.Scatter(x=fdate, y=pressure, mode='lines', name="Pressure"),
    row=3, col=2
)


# weather
fig.add_trace(go.Bar(x=['Rain','Thunder','Fog','Snow','Hail','SeaIce','Gusts'], y=weather[0], showlegend=False),
  row=4, col=2
)

# create sequence of frames 
fig.frames=[go.Frame(
        data=[
            # update map dot
            go.Scattergeo(lat=x, lon=y, mode = 'lines'),
            go.Scattergeo(
            lat=[x[k]],
            lon=[y[k]]),
          
            # update wind force 
            go.Scatter(x=[fdate[k]],y=[wforce[k]], mode='markers'),
            go.Scatter(x=fdate, y=wforce, mode='lines'),
            
            # update temperature 
            go.Scatter(x=[fdate[k]],y=[temperature[k]], mode='markers'),
            go.Scatter(x=fdate, y=temperature, mode='lines'),
            
            # update pressure
            go.Scatter(x=[fdate[k]],y=[pressure[k]], mode='markers'),
            go.Scatter(x=fdate, y=pressure, mode='lines'),
            
            # update weather variables 
            go.Bar(x=['Rain','Thunder','Fog','Snow','Hail','SeaIce','Gusts'], y=weather[k], showlegend=False)
            ],  name=str(k))
        for k in range(coord.shape[0])]


sliders = [
            {
                "steps": [
                   {
                        "args": [[f.name], { "frame": {"duration": 0}, "mode": "update", "redraw": False}],
                        "label": fdate[k].strftime("%d.%m.%Y"),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

fig.update_yaxes(showticklabels=False, row=4, col=2, range=[0,1])
fig.update_xaxes(showticklabels=False, row=1, col=2, range=[fdate[0],fdate[-1]])
fig.update_xaxes(showticklabels=False, row=2, col=2, range=[fdate[0],fdate[-1]])
fig.update_xaxes(showticklabels=False, row=3, col=2, range=[fdate[0],fdate[-1]])

fig.update_layout(
        title_text="Route of ship " + ship_name, 
        hovermode="closest",
        sliders=sliders)
fig.show()